# 🚚 Expansion européenne

### 📖 Histoire

C'est l'année **1956**. Un camion Festo quitte le siège social en Allemagne et doit livrer un envoi à la première entreprise nationale à Milan, en Italie. Le paysage européen est mixte - autoroutes, routes régionales accidentées et cols de montagne. Votre programme doit calculer l'itinéraire le moins coûteux du siège social au site italien.

### 🎯 Objectif

Calculez le chemin le **plus rapide** (le moins coûteux) du siège social de Festo à Festo Italie.

**Mouvements autorisés :** haut, bas, gauche et droite (les mouvements diagonaux ne sont **pas** autorisés).

---

## 📊 Table des symboles et coûts

| Symbole | Signification | Coût |
|:---:|---|---:|
| `.` | Autoroute / route plate | 1 |
| `#` | Obstacle / terrain impassable | ∞ |
| `~` | Route régionale accidentée | 3 |
| `^` | Pass de montagne | 5 |
| `S` | Début (QG de Festo, Allemagne) | 0 |
| `G` | But (Festo Italie) | 0 |

---

## 📋 Exemple

La carte du terrain est donnée sous la forme d'une grille. Chaque symbole représente un type de route ou d'obstacle.

### Entrée

```
S..~...^^.
.#.^..#..G
..#..~~...
..#..#....
..#..#....
..#..#....
..#..#....
..#..#....
..#..#....
..........
```

### Solution

Dans cet exemple, le chemin le plus court (qui est aussi la chaîne de solution) aurait un **coût total de 15** :

```
S..~...^..G
```

---

## 🏆 Défi

Utilisez la carte décrite dans `map.txt` pour trouver le chemin le plus rapide (le moins coûteux) du siège de Festo à Festo en Italie.

# Proposition de résolution

- On lit la grille comme si c´était un vecteur
- On calcule le coût par case
- On détermine des arrêtes avec les 4 voisins


In [29]:
import numpy as np


def lire_grille(fichier):
    with open(fichier, 'r') as f:
        lignes = f.readlines()
    return np.array([list(ligne.rstrip("\n")) for ligne in lignes])

grille = lire_grille('res/map.txt')
H, L = grille.shape
N = H * L
print(grille)
print("Grille dimensions:", grille.shape)
print("Nombre total de cellules:", N)


[['S' '.' '~' ... '.' '^' '.']
 ['.' '.' '#' ... '.' '.' '~']
 ['.' '~' '.' ... '.' '~' '.']
 ...
 ['.' '.' '~' ... '^' '~' '.']
 ['.' '~' '.' ... '#' '.' '.']
 ['.' '.' '^' ... '.' 'G' '.']]
Grille dimensions: (38, 40)
Nombre total de cellules: 1520


In [30]:
# Initialisation de la matrice de coûts
cout = np.full((H, L), np.inf, dtype=np.float32)
# Allocation des coûts par slicing
cout[grille == '.'] = 1 # Autoroute
cout[grille == '~'] = 3 # Route réginonale accidentée
cout[grille == "^"] = 5 # Montagne
cout[(grille == "S") | (grille == "G")] = 0 # Départ et arrivée
print("\nMatrice des coûts:")
print(cout)

# Masque indiquant la practicabilité du chemin
franchissable = np.isfinite(cout)
print("\nMasque de practicabilité:")
print(franchissable)

# Définition des indexes
indexes = np.arange(N).reshape(H, L)
print("\nIndexes des cellules:")
print(indexes)



Matrice des coûts:
[[ 0.  1.  3. ...  1.  5.  1.]
 [ 1.  1. inf ...  1.  1.  3.]
 [ 1.  3.  1. ...  1.  3.  1.]
 ...
 [ 1.  1.  3. ...  5.  3.  1.]
 [ 1.  3.  1. ... inf  1.  1.]
 [ 1.  1.  5. ...  1.  0.  1.]]

Masque de practicabilité:
[[ True  True  True ...  True  True  True]
 [ True  True False ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ... False  True  True]
 [ True  True  True ...  True  True  True]]

Indexes des cellules:
[[   0    1    2 ...   37   38   39]
 [  40   41   42 ...   77   78   79]
 [  80   81   82 ...  117  118  119]
 ...
 [1400 1401 1402 ... 1437 1438 1439]
 [1440 1441 1442 ... 1477 1478 1479]
 [1480 1481 1482 ... 1517 1518 1519]]


In [31]:
franchissable.ravel()[indexes[:-1, :].ravel()]

array([ True,  True,  True, ..., False,  True,  True], shape=(1480,))

In [32]:
indexes[:-1, :].ravel()

array([   0,    1,    2, ..., 1477, 1478, 1479], shape=(1480,))

In [33]:
# Calcule des arrêtes
from typing import Tuple

from scipy import sparse


def arretes(arretes_depart: np.ndarray, arretes_arrivee: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Génère les arrêtes entre deux côtés adjacents de la grille.
    Args:
        arretes_depart (np.ndarray): Indexes des cellules de départ.
        arretes_arrivee (np.ndarray): Indexes des cellules d'arrivée.
    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray]: Trois tableaux contenant les indexes des cellules de départ, 
        les indexes des cellules d'arrivée, et les coûts associés aux cellules d'arrivée.
    """
    depart = arretes_depart.ravel()
    arrivee = arretes_arrivee.ravel()
    # Filtrage des franchissements valides, c´est-à-dire franchissables au départ et à l'arrivée
    m = franchissable.ravel()[depart] & franchissable.ravel()[arrivee]  

    return depart[m], arrivee[m], cout.ravel()[arrivee[m]]


# Réalisation des calculs pour les voisins verticaux
dep_bas, arr_bas, cout_bas = arretes(indexes[:-1, :], indexes[1:, :])  # voisins du bas
dep_haut, arr_haut, cout_haut = arretes(indexes[1:, :], indexes[:-1, :])  # voisins du haut

# Réalisation des calculs pour les voisins horizontaux
dep_droite, arr_droite, cout_droite = arretes(indexes[:, :-1], indexes[:, 1:])  # voisins de droite
dep_gauche, arr_gauche, cout_gauche = arretes(indexes[:, 1:], indexes[:, :-1])  # voisins de gauche

lignes = np.concatenate([dep_bas, dep_haut, dep_droite, dep_gauche])
colonnes = np.concatenate([arr_bas, arr_haut, arr_droite, arr_gauche])
couts = np.concatenate([cout_bas, cout_haut, cout_droite, cout_gauche])

G = sparse.csr_matrix((couts, (lignes, colonnes)), shape=(N, N))
G


<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 4992 stored elements and shape (1520, 1520)>

In [34]:
# Definition de l´origine et de la destination
s = int(np.flatnonzero((grille == 'S').ravel())[0])
g = int(np.flatnonzero((grille == 'G').ravel())[0])

print("Index de départ (S):", s)
print("Index de destination (G):", g)

Index de départ (S): 0
Index de destination (G): 1518


In [35]:
# Calcul du chemin le plus court

from scipy.sparse import csgraph
distance, predecessors = csgraph.dijkstra(G, directed=True, indices=s, return_predecessors=True)
meilleur_cout = distance[g]
print("Coût du meilleur chemin de S à G:", meilleur_cout)

Coût du meilleur chemin de S à G: 102.0


In [36]:
# Reconstruction du chemin optimal
chemin = []
position_actuelle = g

while position_actuelle != s:
    chemin.append(position_actuelle)
    position_actuelle = predecessors[position_actuelle]
chemin.append(s)
chemin.reverse()

# Conversion des indexes en succession des symboles de la grille
chemin_symboles = ''.join([grille[divmod(int(index), L)] for index in chemin])
print("Chemin optimal de S à G  --> ", chemin_symboles)

Chemin optimal de S à G  -->  S............~....~.....~......~..~........~.....~......~..~........~..^^..G
